In [37]:
import pyspark
from datetime import datetime
from pyspark.sql.functions import *

In [2]:
pyspark.__version__

'3.3.1'

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("test") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/25 15:12:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
!ls

Untitled.ipynb                test.ipynb
fhvhv_tripdata_2021-06.csv.gz


In [21]:
df = spark.read.option('inferSchema', True).option('header', True).csv('fhvhv_tripdata_2021-06.csv.gz')

In [15]:
df = df.repartition(12)

In [17]:
df.write.parquet('output')

23/02/25 15:28:28 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [19]:
!ls -lh output

total 615936
-rw-r--r--  1 laede  staff     0B Feb 25 15:28 _SUCCESS
-rw-r--r--  1 laede  staff    25M Feb 25 15:28 part-00000-7faf5852-7f35-4b5c-8941-14da19e474ae-c000.snappy.parquet
-rw-r--r--  1 laede  staff    25M Feb 25 15:28 part-00001-7faf5852-7f35-4b5c-8941-14da19e474ae-c000.snappy.parquet
-rw-r--r--  1 laede  staff    25M Feb 25 15:28 part-00002-7faf5852-7f35-4b5c-8941-14da19e474ae-c000.snappy.parquet
-rw-r--r--  1 laede  staff    25M Feb 25 15:28 part-00003-7faf5852-7f35-4b5c-8941-14da19e474ae-c000.snappy.parquet
-rw-r--r--  1 laede  staff    25M Feb 25 15:28 part-00004-7faf5852-7f35-4b5c-8941-14da19e474ae-c000.snappy.parquet
-rw-r--r--  1 laede  staff    25M Feb 25 15:28 part-00005-7faf5852-7f35-4b5c-8941-14da19e474ae-c000.snappy.parquet
-rw-r--r--  1 laede  staff    25M Feb 25 15:28 part-00006-7faf5852-7f35-4b5c-8941-14da19e474ae-c000.snappy.parquet
-rw-r--r--  1 laede  staff    25M Feb 25 15:28 part-00007-7faf5852-7f35-4b5c-8941-14da19e474ae-c000.snappy.parquet
-rw-r--r-- 

In [22]:
display(df)

DataFrame[dispatching_base_num: string, pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: int, DOLocationID: int, SR_Flag: string, Affiliated_base_number: string]

In [23]:
df.show(10)

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02764|2021-06-01 00:02:41|2021-06-01 00:07:46|         174|          18|      N|                B02764|
|              B02764|2021-06-01 00:16:16|2021-06-01 00:21:14|          32|         254|      N|                B02764|
|              B02764|2021-06-01 00:27:01|2021-06-01 00:42:11|         240|         127|      N|                B02764|
|              B02764|2021-06-01 00:46:08|2021-06-01 00:53:45|         127|         235|      N|                B02764|
|              B02510|2021-06-01 00:45:42|2021-06-01 01:03:33|         144|         146|      N|                  null|
|              B02510|2021-06-01 00:18:1

In [26]:
 df_june_15 = df.filter((df.pickup_datetime >= datetime(2021, 6, 15)) & (df.pickup_datetime < datetime(2021, 6, 16)))

In [27]:
df_june_15.count()

452470

In [38]:
df = df.withColumn('duration_sec', unix_timestamp('dropoff_datetime') - unix_timestamp('pickup_datetime'))

In [39]:
max_duration_sec = df.select(max(df.duration_sec))

In [40]:
max_duration_hour = max_duration_sec.collect()[0][0] / 3600

In [41]:
max_duration_hour

66.8788888888889

In [43]:
!ls -lh

total 361096
-rw-r--r--@  1 laede  staff   168M Feb 25 14:01 fhvhv_tripdata_2021-06.csv.gz
drwxr-xr-x  28 laede  staff   896B Feb 25 15:28 output
-rw-r--r--   1 laede  staff    10K Feb 25 15:57 spark.ipynb
-rw-r--r--@  1 laede  staff    12K Feb 25 16:00 taxi_zone_lookup.csv
-rw-r--r--   1 laede  staff    40K Jan 27 11:55 test.ipynb


In [44]:
zones = spark.read.option('inferSchema', True).option('header', True).csv('taxi_zone_lookup.csv')

In [47]:
display(zones)

DataFrame[LocationID: int, Borough: string, Zone: string, service_zone: string]

In [46]:
joined = df.join(zones, df.PULocationID == zones.LocationID)

In [48]:
joined.createOrReplaceTempView("trips")

In [50]:
display(joined)

DataFrame[dispatching_base_num: string, pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: int, DOLocationID: int, SR_Flag: string, Affiliated_base_number: string, duration: interval day to second, duration_sec: bigint, LocationID: int, Borough: string, Zone: string, service_zone: string]

In [51]:
spark.sql('select zone, count(*) as cnt from trips group by zone order by 2 desc').show()

+--------------------+------+
|                zone|   cnt|
+--------------------+------+
| Crown Heights North|231279|
|        East Village|221244|
|         JFK Airport|188867|
|      Bushwick South|187929|
|       East New York|186780|
|TriBeCa/Civic Center|164344|
|   LaGuardia Airport|161596|
|            Union Sq|158937|
|        West Village|154698|
|             Astoria|152493|
|     Lower East Side|151020|
|        East Chelsea|147673|
|Central Harlem North|146402|
|Williamsburg (Nor...|143683|
|          Park Slope|143594|
|  Stuyvesant Heights|141427|
|        Clinton East|139611|
|West Chelsea/Huds...|139431|
|             Bedford|138428|
|         Murray Hill|137879|
+--------------------+------+
only showing top 20 rows

